# Battery Data Tool - Button Functions with Real Data

This notebook demonstrates button functions using actual Rawdata from the battery testing system.

**Data Path**: `c:\Users\Ryu\Python_project\data\battery251027\Rawdata`

## Setup & Imports

In [ ]:
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path

# Add parent directory to import BatteryDataTool functions
sys.path.append(r'c:\Users\Ryu\Python_project\data\battery251027')

# Import helper functions from BatteryDataTool.py
from BatteryDataTool import (
    check_cycler,
    pne_cycle_data,
    toyo_cycle_data,
    pne_step_Profile_data,
    toyo_step_Profile_data,
    pne_rate_Profile_data,
    toyo_rate_Profile_data,
    pne_chg_Profile_data,
    toyo_chg_Profile_data,
    pne_dchg_Profile_data,
    toyo_dchg_Profile_data,
    pne_Profile_continue_data,
    pne_dcir_Profile_data
)

# Configuration
plt.rcParams['figure.figsize'] = (14, 8)
plt.rcParams['font.size'] = 10
plt.rcParams['font.family'] = 'Malgun Gothic'  # Korean font support

# Base path for raw data
RAWDATA_PATH = r'c:\Users\Ryu\Python_project\data\battery251027\Rawdata'

# Color palette
COLORS = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', 
          '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf']

print("✅ Setup complete")
print(f"📁 Rawdata path: {RAWDATA_PATH}")
print(f"📊 Available test folders: {len(os.listdir(RAWDATA_PATH))}")

## Explore Available Data

In [ ]:
# List all test folders
test_folders = [f for f in os.listdir(RAWDATA_PATH) if os.path.isdir(os.path.join(RAWDATA_PATH, f))]

print("Available Test Folders:")
print("=" * 80)
for idx, folder in enumerate(test_folders, 1):
    folder_path = os.path.join(RAWDATA_PATH, folder)
    is_pne = check_cycler(folder_path)
    cycler_type = "PNE" if is_pne else "TOYO"
    
    # Count channels
    channels = [f for f in os.listdir(folder_path) if os.path.isdir(os.path.join(folder_path, f)) and 'Ch' in f]
    
    print(f"{idx:2d}. [{cycler_type:4s}] {folder} ({len(channels)} channels)")

print("=" * 80)

## Example 1: Individual Cycle Analysis

Analyze cycle aging data from A1_MP1_4500mAh_T23_1 test

In [ ]:
def indiv_cyc_analysis_real(test_folder_name, mincapacity=4500, firstCrate=0.5):
    """
    Individual Cycle Analysis with Real Data
    
    Parameters:
        test_folder_name: str - name of test folder in Rawdata
        mincapacity: float - minimum capacity in mAh
        firstCrate: float - first C-rate for capacity calculation
    """
    test_path = os.path.join(RAWDATA_PATH, test_folder_name)
    
    if not os.path.exists(test_path):
        print(f"❌ Error: {test_folder_name} not found")
        return None
    
    # Check cycler type
    is_pne = check_cycler(test_path)
    print(f"📊 Analyzing: {test_folder_name}")
    print(f"🔧 Cycler type: {'PNE' if is_pne else 'TOYO'}")
    
    # Get channel folders
    channels = [f for f in os.listdir(test_path) 
                if os.path.isdir(os.path.join(test_path, f)) and 'Ch' in f]
    
    print(f"📁 Found {len(channels)} channels")
    
    results = []
    
    for ch_idx, channel in enumerate(channels[:2]):  # Analyze first 2 channels
        channel_path = os.path.join(test_path, channel)
        print(f"\n  Processing: {channel}...")
        
        try:
            # Load cycle data using actual BatteryDataTool function
            if is_pne:
                capacity, cycle_obj = pne_cycle_data(
                    channel_path, 
                    mincapacity, 
                    firstCrate, 
                    chkir=False,  # DCIR check
                    chkir2=False,  # DCIR2 check
                    mkdcir=False   # Make DCIR
                )
            else:
                capacity, cycle_obj = toyo_cycle_data(
                    channel_path,
                    mincapacity,
                    firstCrate,
                    chkir=False
                )
            
            if not hasattr(cycle_obj, 'NewData'):
                print(f"  ⚠️  No cycle data available for {channel}")
                continue
            
            df = cycle_obj.NewData
            print(f"  ✅ Loaded {len(df)} cycles, Capacity: {capacity:.1f} mAh")
            
            # Create visualization
            fig, axes = plt.subplots(2, 3, figsize=(14, 8))
            cycles = df.index.values
            
            # Panel 1: Discharge Capacity
            if '방전용량' in df.columns:
                axes[0, 0].plot(cycles, df['방전용량'], 'o-', color=COLORS[0], markersize=3)
                axes[0, 0].set_ylabel('Dchg Capacity (mAh)')
                axes[0, 0].set_xlabel('Cycle')
                axes[0, 0].grid(True, alpha=0.3)
            
            # Panel 2: Charge Capacity
            if '충전용량' in df.columns:
                axes[0, 1].plot(cycles, df['충전용량'], 'o-', color=COLORS[1], markersize=3)
                axes[0, 1].set_ylabel('Chg Capacity (mAh)')
                axes[0, 1].set_xlabel('Cycle')
                axes[0, 1].grid(True, alpha=0.3)
            
            # Panel 3: Efficiency
            if '충방효율' in df.columns:
                axes[0, 2].plot(cycles, df['충방효율'], 'o-', color=COLORS[2], markersize=3)
                axes[0, 2].set_ylabel('Efficiency (%)')
                axes[0, 2].set_xlabel('Cycle')
                axes[0, 2].grid(True, alpha=0.3)
            
            # Panel 4: Rest Voltage
            if 'Rest End' in df.columns:
                axes[1, 0].plot(cycles, df['Rest End'], 'o-', color=COLORS[3], markersize=3)
                axes[1, 0].set_ylabel('Rest Voltage (V)')
                axes[1, 0].set_xlabel('Cycle')
                axes[1, 0].grid(True, alpha=0.3)
            
            # Panel 5: DCIR (if available)
            if 'dcir' in df.columns:
                dcir_data = df['dcir'].dropna()
                if len(dcir_data) > 0:
                    axes[1, 1].plot(dcir_data.index, dcir_data.values, 'o-', color=COLORS[4], markersize=3)
                    axes[1, 1].set_ylabel('DCIR (mΩ)')
                    axes[1, 1].set_xlabel('Cycle')
                    axes[1, 1].grid(True, alpha=0.3)
            
            # Panel 6: Energy
            if '방전Energy' in df.columns:
                axes[1, 2].plot(cycles, df['방전Energy'], 'o-', color=COLORS[5], markersize=3)
                axes[1, 2].set_ylabel('Dchg Energy (Wh)')
                axes[1, 2].set_xlabel('Cycle')
                axes[1, 2].grid(True, alpha=0.3)
            
            plt.suptitle(f'{test_folder_name} - {channel}', fontsize=15, fontweight='bold')
            plt.tight_layout()
            
            results.append({
                'channel': channel,
                'capacity': capacity,
                'data': df,
                'figure': fig
            })
            
        except Exception as e:
            print(f"  ❌ Error processing {channel}: {str(e)}")
    
    return results

# Run analysis
results = indiv_cyc_analysis_real('A1_MP1_4500mAh_T23_1', mincapacity=4500, firstCrate=0.5)
plt.show()

## Example 2: Step Profile Analysis

Analyze charging step profiles at specific cycles

In [ ]:
def step_profile_analysis_real(test_folder_name, cycle_numbers=[1, 50, 100], 
                                mincapacity=4500, firstCrate=0.5, mincrate=0.05):
    """
    Step Profile Analysis with Real Data
    
    Parameters:
        test_folder_name: str - name of test folder
        cycle_numbers: list - cycle numbers to analyze
        mincapacity: float - minimum capacity
        firstCrate: float - first C-rate
        mincrate: float - minimum C-rate threshold
    """
    test_path = os.path.join(RAWDATA_PATH, test_folder_name)
    
    if not os.path.exists(test_path):
        print(f"❌ Error: {test_folder_name} not found")
        return None
    
    is_pne = check_cycler(test_path)
    print(f"📊 Step Profile Analysis: {test_folder_name}")
    print(f"🔧 Cycler: {'PNE' if is_pne else 'TOYO'}")
    print(f"📈 Cycles to analyze: {cycle_numbers}")
    
    channels = [f for f in os.listdir(test_path) 
                if os.path.isdir(os.path.join(test_path, f)) and 'Ch' in f]
    
    for channel in channels[:1]:  # Analyze first channel
        channel_path = os.path.join(test_path, channel)
        print(f"\n  Processing: {channel}")
        
        fig, axes = plt.subplots(2, 3, figsize=(14, 10))
        
        for cyc_no in cycle_numbers:
            try:
                # Load step profile data
                if is_pne:
                    capacity, profile_obj = pne_step_Profile_data(
                        channel_path, cyc_no, mincapacity, mincrate, firstCrate
                    )
                else:
                    capacity, profile_obj = toyo_step_Profile_data(
                        channel_path, cyc_no, mincapacity, mincrate, firstCrate
                    )
                
                if not hasattr(profile_obj, 'stepchg') or len(profile_obj.stepchg) < 2:
                    print(f"  ⚠️  No step data for cycle {cyc_no}")
                    continue
                
                df = profile_obj.stepchg
                label = f"Cycle {cyc_no:04d}"
                print(f"  ✅ Cycle {cyc_no}: {len(df)} points")
                
                # Plot 6 panels
                axes[0, 0].plot(df['TimeMin'], df['Vol'], label=label)
                axes[0, 1].plot(df['TimeMin'], df['Vol'], label=label)
                axes[0, 2].plot(df['TimeMin'], df['Vol'], label=label)
                axes[1, 0].plot(df['TimeMin'], df['SOC'], label=label)
                axes[1, 1].plot(df['TimeMin'], df['Crate'], label=label)
                axes[1, 2].plot(df['TimeMin'], df['Temp'], label=label)
                
            except Exception as e:
                print(f"  ❌ Error at cycle {cyc_no}: {str(e)}")
        
        # Configure axes
        axes[0, 0].set_ylabel('Voltage (V)')
        axes[0, 1].set_ylabel('Voltage (V)')
        axes[0, 2].set_ylabel('Voltage (V)')
        axes[1, 0].set_ylabel('SOC')
        axes[1, 1].set_ylabel('C-rate')
        axes[1, 2].set_ylabel('Temperature (°C)')
        
        for ax in axes.flatten():
            ax.set_xlabel('Time (min)')
            ax.legend()
            ax.grid(True, alpha=0.3)
        
        plt.suptitle(f'Step Profile - {test_folder_name} - {channel}', 
                    fontsize=15, fontweight='bold')
        plt.tight_layout()

# Run analysis
step_profile_analysis_real('A1_MP1_4500mAh_T23_1', cycle_numbers=[1, 10, 50])
plt.show()

## Example 3: Charge Profile with dQ/dV Analysis

In [ ]:
def charge_profile_analysis_real(test_folder_name, cycle_numbers=[1, 50, 100],
                                  mincapacity=4500, firstCrate=0.5, 
                                  mincrate=0.05, smoothdegree=5):
    """
    Charge Profile Analysis with dQ/dV
    """
    test_path = os.path.join(RAWDATA_PATH, test_folder_name)
    
    if not os.path.exists(test_path):
        print(f"❌ Error: {test_folder_name} not found")
        return None
    
    is_pne = check_cycler(test_path)
    print(f"📊 Charge Profile Analysis: {test_folder_name}")
    print(f"🔧 Cycler: {'PNE' if is_pne else 'TOYO'}")
    
    channels = [f for f in os.listdir(test_path) 
                if os.path.isdir(os.path.join(test_path, f)) and 'Ch' in f]
    
    for channel in channels[:1]:
        channel_path = os.path.join(test_path, channel)
        print(f"\n  Processing: {channel}")
        
        fig, axes = plt.subplots(2, 3, figsize=(14, 10))
        
        for cyc_no in cycle_numbers:
            try:
                if is_pne:
                    capacity, profile_obj = pne_chg_Profile_data(
                        channel_path, cyc_no, mincapacity, mincrate, firstCrate, smoothdegree
                    )
                else:
                    capacity, profile_obj = toyo_chg_Profile_data(
                        channel_path, cyc_no, mincapacity, mincrate, firstCrate, smoothdegree
                    )
                
                if not hasattr(profile_obj, 'Profile') or len(profile_obj.Profile) < 2:
                    print(f"  ⚠️  No profile data for cycle {cyc_no}")
                    continue
                
                df = profile_obj.Profile
                label = f"Cycle {cyc_no:04d}"
                print(f"  ✅ Cycle {cyc_no}: {len(df)} points")
                
                # 6 panels: SOC-V (2x), dQ/dV, dV/dQ, C-rate, Temp
                axes[0, 0].plot(df['SOC'], df['Vol'], label=label)
                axes[0, 1].plot(df['Vol'], df['dQdV'], label=label)
                axes[0, 2].plot(df['SOC'], df['Vol'], label=label)
                axes[1, 0].plot(df['SOC'], df['dVdQ'], label=label)
                axes[1, 1].plot(df['SOC'], df['Crate'], label=label)
                axes[1, 2].plot(df['SOC'], df['Temp'], label=label)
                
            except Exception as e:
                print(f"  ❌ Error at cycle {cyc_no}: {str(e)}")
        
        # Configure axes
        axes[0, 0].set_xlabel('SOC')
        axes[0, 0].set_ylabel('Voltage (V)')
        axes[0, 1].set_xlabel('Voltage (V)')
        axes[0, 1].set_ylabel('dQ/dV')
        axes[0, 2].set_xlabel('SOC')
        axes[0, 2].set_ylabel('Voltage (V)')
        axes[1, 0].set_xlabel('SOC')
        axes[1, 0].set_ylabel('dV/dQ')
        axes[1, 1].set_xlabel('SOC')
        axes[1, 1].set_ylabel('C-rate')
        axes[1, 2].set_xlabel('SOC')
        axes[1, 2].set_ylabel('Temperature (°C)')
        
        for ax in axes.flatten():
            ax.legend()
            ax.grid(True, alpha=0.3)
        
        plt.suptitle(f'Charge Profile (dQ/dV) - {test_folder_name} - {channel}',
                    fontsize=15, fontweight='bold')
        plt.tight_layout()

# Run analysis
charge_profile_analysis_real('A1_MP1_4500mAh_T23_1', cycle_numbers=[1, 50])
plt.show()

## Example 4: DCIR Analysis (PNE Only)

In [ ]:
def dcir_analysis_real(test_folder_name, cycle_range='1-10', 
                       mincapacity=4500, firstCrate=0.5):
    """
    DCIR Analysis (PNE only)
    """
    test_path = os.path.join(RAWDATA_PATH, test_folder_name)
    
    if not os.path.exists(test_path):
        print(f"❌ Error: {test_folder_name} not found")
        return None
    
    is_pne = check_cycler(test_path)
    if not is_pne:
        print(f"❌ DCIR analysis requires PNE cycler data")
        return None
    
    print(f"📊 DCIR Analysis: {test_folder_name}")
    print(f"📈 Cycle range: {cycle_range}")
    
    # Parse cycle range
    start_cyc, end_cyc = map(int, cycle_range.split('-'))
    
    channels = [f for f in os.listdir(test_path) 
                if os.path.isdir(os.path.join(test_path, f)) and 'Ch' in f]
    
    for channel in channels[:1]:
        channel_path = os.path.join(test_path, channel)
        print(f"\n  Processing: {channel}")
        
        try:
            result = pne_dcir_Profile_data(
                channel_path, start_cyc, end_cyc, mincapacity, firstCrate
            )
            
            if result is None or not hasattr(result[1], 'SOC'):
                print(f"  ⚠️  No DCIR data available")
                continue
            
            capacity, df = result
            print(f"  ✅ DCIR data: {len(df)} points")
            
            # Create 4-panel figure
            fig, axes = plt.subplots(2, 2, figsize=(14, 8))
            
            # Panel 1: SOC vs OCV/CCV
            axes[0, 0].plot(df['SOC'], df['OCV'], 'o-', label='OCV', markersize=6)
            if 'rOCV' in df.columns:
                axes[0, 0].plot(df['SOC'], df['rOCV'], 's-', label='rOCV', markersize=6)
            axes[0, 0].plot(df['SOC'], df['CCV'], '^-', label='CCV', markersize=6)
            axes[0, 0].set_xlabel('SOC (%)')
            axes[0, 0].set_ylabel('Voltage (V)')
            axes[0, 0].set_title('OCV/CCV vs SOC')
            axes[0, 0].legend()
            axes[0, 0].grid(True, alpha=0.3)
            
            # Panel 2: SOC vs DCIR
            dcir_cols = [col for col in df.columns if 'DCIR' in col or col == 'RSS']
            for col in dcir_cols:
                axes[0, 1].plot(df['SOC'], df[col], 'o-', label=col, markersize=4)
            axes[0, 1].set_xlabel('SOC (%)')
            axes[0, 1].set_ylabel('DCIR (mΩ)')
            axes[0, 1].set_title('DCIR vs SOC')
            axes[0, 1].legend()
            axes[0, 1].grid(True, alpha=0.3)
            
            # Panel 3: Voltage vs SOC
            axes[1, 0].plot(df['OCV'], df['SOC'], 'o-', label='OCV', markersize=6)
            axes[1, 0].plot(df['CCV'], df['SOC'], '^-', label='CCV', markersize=6)
            axes[1, 0].set_xlabel('Voltage (V)')
            axes[1, 0].set_ylabel('SOC (%)')
            axes[1, 0].set_title('SOC vs Voltage')
            axes[1, 0].legend()
            axes[1, 0].grid(True, alpha=0.3)
            
            # Panel 4: OCV vs DCIR
            for col in dcir_cols:
                axes[1, 1].plot(df['OCV'], df[col], 'o-', label=col, markersize=4)
            axes[1, 1].set_xlabel('OCV (V)')
            axes[1, 1].set_ylabel('DCIR (mΩ)')
            axes[1, 1].set_title('DCIR vs OCV')
            axes[1, 1].legend()
            axes[1, 1].grid(True, alpha=0.3)
            
            plt.suptitle(f'DCIR Analysis - {test_folder_name} - {channel} (Cycles {start_cyc}-{end_cyc})',
                        fontsize=15, fontweight='bold')
            plt.tight_layout()
            
            # Display summary
            print(f"\n  📊 DCIR Summary:")
            print(f"     Capacity: {capacity:.1f} mAh")
            print(f"     SOC range: {df['SOC'].min():.1f}% - {df['SOC'].max():.1f}%")
            if 'RSS' in df.columns:
                print(f"     RSS DCIR: {df['RSS'].min():.2f} - {df['RSS'].max():.2f} mΩ")
            
        except Exception as e:
            print(f"  ❌ Error: {str(e)}")

# Run analysis on PNE data
# Note: Use a test folder that has DCIR test data
# dcir_analysis_real('Q7M Inner ATL_45V 1689mAh BLK1 20EA [23] - 250304', cycle_range='1-10')
# plt.show()

print("✅ DCIR analysis function ready (requires PNE cycler with DCIR test data)")

## Data Export Example

In [ ]:
# Export analyzed data to Excel
def export_cycle_data(results, output_filename='battery_analysis.xlsx'):
    """
    Export analysis results to Excel
    
    Parameters:
        results: list of result dicts from analysis functions
        output_filename: str - output Excel filename
    """
    if not results:
        print("❌ No results to export")
        return
    
    output_path = os.path.join(RAWDATA_PATH, '..', output_filename)
    
    with pd.ExcelWriter(output_path, engine='xlsxwriter') as writer:
        for idx, result in enumerate(results):
            sheet_name = f"{result['channel'][:25]}"  # Excel sheet name limit
            result['data'].to_excel(writer, sheet_name=sheet_name, index=True)
            print(f"  ✅ Exported: {sheet_name}")
    
    print(f"\n📄 Saved to: {output_path}")

# Example usage:
# if results:
#     export_cycle_data(results, 'A1_MP1_cycle_analysis.xlsx')

print("✅ Export function ready")

## Quick Test Summary

In [ ]:
print("""
═══════════════════════════════════════════════════════════════
  Battery Data Tool - Real Data Analysis Summary
═══════════════════════════════════════════════════════════════

✅ Available Functions:

1. indiv_cyc_analysis_real(folder_name, mincapacity, firstCrate)
   → Individual cycle aging analysis with 6-panel visualization

2. step_profile_analysis_real(folder_name, cycle_numbers, ...)
   → Step-by-step charging profile analysis

3. charge_profile_analysis_real(folder_name, cycle_numbers, ...)
   → Charge profile with dQ/dV differential analysis

4. dcir_analysis_real(folder_name, cycle_range, ...) [PNE only]
   → DC Internal Resistance analysis

5. export_cycle_data(results, output_filename)
   → Export analyzed data to Excel

═══════════════════════════════════════════════════════════════
📁 Data Location: c:\\Users\\Ryu\\Python_project\\data\\battery251027\\Rawdata
═══════════════════════════════════════════════════════════════
""")